# Shor's factoring algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's general factoring algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and with the classical post-processing in [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and [[E22p]](https://doi.org/10.48550/arXiv.2201.07791).

To start off, let us pick $n$ prime factors $p_i$ uniformly at random from the set of all $l$-bit prime factors, and $n$ exponents $e_i$ uniformly at random from $[1, e_{\max}] \cap \mathbb Z$, for $i \in [1, n] \cap \mathbb Z$, and take the product to form

$$N = \prod_{i=1}^n p_i^{e_i}.$$

To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) and [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [ ]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [50]:
from quaspy.math.primes import sample_l_bit_prime;
from quaspy.math.random import sample_integer;

from math import prod;

n = 10;
l = 1024;
e_max = 1;

factors = [];
primes = set();

while len(factors) < n:
  pi = sample_l_bit_prime(l);
  ei = sample_integer(e_max) + 1;

  if pi not in primes:
    primes.add(pi);
    factors.append([pi, ei]);

    print("Sampled p" + str(len(factors)) + " =", pi);

N = prod([pi ** ei for [pi, ei] in factors]);

print("\nComputed N =", N);

Sampled p1 = 101127457083216239607260037524508344625763861962136625242530979023750981716793831621986469519497221632194757946167863568609241093392897486016883574305709475063291084824697893812128808590242719288284160394090553689881780768452386741883917392079192307844818134464666938109877085455288309093554788547515251398057
Sampled p2 = 95642517997049540077507871019623337654990291762469900998794020741914418223226219484914512729851833688353703190601239455209547062433116939751007448890615267998873382329217199695921214528065950148996026845185422317190380999890225333854764677525659185445018259171781062372928454310392138160461947896658681809423
Sampled p3 = 109464431735294159974378294274525657206318819420403729824301732669900016145525461205808884543364511930776254665221392803793527815537572044604556948966542735803855527632204173348346705081859987947224360357277306159043226383759325438531440117941819331729497009514666791332843453086877496404634366210511817347747
Sampled p4 = 1377342937868489973

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$. This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$.

In [51]:
from quaspy.factoring.sampling import sample_r_given_N;

r = sample_r_given_N(N, factors);

print("Sampled r =", r);

Sampled r = 1068707828986540595224111100942470531125461379563831226738174603966336749632074703293981755085791586686358265496133947619041361420431631822294396350239983282313467240814588916753624806580343966661695746091425869176908380228395759623807425930379826037154983672125277010219012318567022602172075065168049835506832429164186945264392603377096524368711893934241676093297472130046577030329065774316200480379263835507350227436179825333972010634191252100279422702567296475794900041115605033597809914117234596890068032617030763776856930423615858136273240781604861415787689792033954209922398544595900425028514401672550773249685296433750455013918138984000077532384204088127601087363164735032155476790563073004255037884957252580316783641407284736180439709036493462343394912701231560224087397293782065933493493176145616990038854437387237841642152318899464413966656981873204924610060653840113817628609978554016939324419527707829594762890889986650849966902222759752557021331858045594691847914620964366183

### 1.1. Solving $r$ for the complete factorization of $N$
We may then use the [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $r$ for the factors of $N$ by using the post-processing from [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [52]:
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

result = solve_r_for_factors(r, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] The integer N was not completely factored.");

Solving for the prime factors yielded: {106440120364005540555351960306079290814562292731728072991643884321484281978314070705417464297190297513041026042299028959463072893825414003286960419655510762043967268690989062795423893615967015042769167807410938369445762446700441936094341844739246701491961360505234382152206390825577848765149813266698206071501, 109464431735294159974378294274525657206318819420403729824301732669900016145525461205808884543364511930776254665221392803793527815537572044604556948966542735803855527632204173348346705081859987947224360357277306159043226383759325438531440117941819331729497009514666791332843453086877496404634366210511817347747, 167226938706190836553643155843119208197862054995067415100217554047405801743210621031327509390733457696587026428066379600702360290229750029561230959243313199878214139188992093824931076608557202587723520663649688259097308452343733278018286029658158983822856538007385804379149382950315116619472385431466470845443, 165951452874233136181136823

## 2. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [53]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 1572452007858526389488344233912828411553928649351983490281468860271646879831443193825079743902947456253987652671950401677327496312971248683118822537441006825617147054167578948201878799368750244495340769902628780896086970319987106909235223670403074385200389557908770886449862973482781711829492155545697291699830535070476500806377118159794599188993800887308827518388679585893072695513398165220119557020863468386631531754941044665026877799108330467209594315298072664687114374409846787213507695955969808983936580524911164124305495064016148451459986979783129042310211483536734035429334729755720681547821418535727073701378554379722507473765378509854033657628085470036912087502570180022208839236284760438639705911852970950129942188098941187508618354177685266738113964375429743262918929586381257871835832236164254238134766253114759570275656953274038337579347538730564725986225392558848095301273161983932904711516273675035379006454804289872510911529400326462806328951121655893408360754696250822202

### 2.1. Solving $j$ for the complete factorization of $N$
We now proceed to solve $j$ for the factors of $N$.

To this end, we first use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$:

In [54]:
from quaspy.math.groups import SimulatedCyclicGroupElement;

g = SimulatedCyclicGroupElement(r);

We may then e.g. use the [<code>solve_j_for_factors(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [55]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors;

result = solve_j_for_factors(j, m, l, g, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] Failed to completely factor the integer N.");

Solving for the prime factors yielded: {106440120364005540555351960306079290814562292731728072991643884321484281978314070705417464297190297513041026042299028959463072893825414003286960419655510762043967268690989062795423893615967015042769167807410938369445762446700441936094341844739246701491961360505234382152206390825577848765149813266698206071501, 109464431735294159974378294274525657206318819420403729824301732669900016145525461205808884543364511930776254665221392803793527815537572044604556948966542735803855527632204173348346705081859987947224360357277306159043226383759325438531440117941819331729497009514666791332843453086877496404634366210511817347747, 167226938706190836553643155843119208197862054995067415100217554047405801743210621031327509390733457696587026428066379600702360290229750029561230959243313199878214139188992093824931076608557202587723520663649688259097308452343733278018286029658158983822856538007385804379149382950315116619472385431466470845443, 165951452874233136181136823

We may also solve in two steps, by first calling [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and by then calling [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) to solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [56]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

rp = solve_j_for_r(j, m, l, g, accept_multiple=True);

print("Solving j for a multiple r' of the order r yielded:", rp);

if (rp != 1) and (r % rp == 0):
  print("[ OK ] A multiple r' of the order r was successfully recovered.\n");

  result = solve_r_for_factors(r, N);

  print("Solving r for the complete factorization of N yielded:", result);

  if result == primes:
    print("[ OK ] All prime factors of N were successfully recovered.");
  else:
    print("[FAIL] Failed to completely factor N.");

else:
  print("[FAIL] Failed to recover a multiple r' of the order r.");

Solving j for a multiple r' of the order r yielded: 106870782898654059522411110094247053112546137956383122673817460396633674963207470329398175508579158668635826549613394761904136142043163182229439635023998328231346724081458891675362480658034396666169574609142586917690838022839575962380742593037982603715498367212527701021901231856702260217207506516804983550683242916418694526439260337709652436871189393424167609329747213004657703032906577431620048037926383550735022743617982533397201063419125210027942270256729647579490004111560503359780991411723459689006803261703076377685693042361585813627324078160486141578768979203395420992239854459590042502851440167255077324968529643375045501391813898400007753238420408812760108736316473503215547679056307300425503788495725258031678364140728473618043970903649346234339491270123156022408739729378206593349349317614561699003885443738723784164215231889946441396665698187320492461006065384011381762860997855401693932441952770782959476289088998665084996690222275975

## 3. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.

This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$. (If the function is also fed the factorization of $p_i - 1$ for $i \in [1, n] \cap \mathbb Z$ then the function can be made exact, as explained in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [57]:
from quaspy.factoring.sampling import sample_g_r_given_N;

[g, r] = sample_g_r_given_N(N, factors);

print("Sampled g =", g, "\n");
print("Sampled r =", r, "\n");

Sampled g = 3367602546893238770512458603279537031021370714301678513306753792537259635700889482382349589469594899061853161641255474400856200926890963051518825674477955214704805137436412333739324476181732716838436521859187576953790026089337606024600449699524346970983752739194741976358965712698258324788088401824888426264571706519315182029837381293019361636872663176712903033052323630560009595074270700699889105476938543832620509633774634020447304950403026132965839984542387397790349701950325237479229223083664092449490243075482217718965942183410133553006110862797442581996958648603294151861328069366528039773200999300417741214249274798958525266415894161156440733881512240170870994657659765588331501356082056628348251377561793378714125141266932794788269053725061482116859082771283232750787836314792581314118059705817371095728880223478230290492765157193719604921044840007164666419650324503967217030641314813290114062410979738159886931140886985252570195066770382015242266355463516341341460954741456032528

### 3.1. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [58]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 1705858917418517588125872085303222263660769669274087967573076299383416234578425819919638067663346653969480865051505648196237395041788413400606536546472235024533659271594511447506016608309977636862614117181152974178017218546285298800892440179541549458016986407289037525820424067930959885357269108840170989988052536367039582859949409410526573962425730532604239952805211276932100213204430178538080846603495606480502726289428882650317004057862114266688659759293673854301182978623237753693214704812466259441222879153897469261138741841847187816586805917609577355851253288912395061017223678814058857647738732664971014755398684912651255356851238710746389074508311360791414453192725683022513483132240751262583573208130574065122213262893354200419298765653997209051325331569384036132907486245342916823374470602515578627115760068903364636875285960444181608882338800659175494625210327844762808673199035722828135918688252334398478233043940702649827485592297140232937430098981661901370571082366274454788

### 3.2. Solving $j$ for the complete factorization of $N$
We may then e.g. use the [<code>solve_j_for_factors_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [59]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors_mod_N;

result = solve_j_for_factors_mod_N(j, m, l, g, N);

print("Solving for all prime factors of N via [E21b] yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] Failed to completely factor the integer N.");

Solving for all prime factors of N via [E21b] yielded: {106440120364005540555351960306079290814562292731728072991643884321484281978314070705417464297190297513041026042299028959463072893825414003286960419655510762043967268690989062795423893615967015042769167807410938369445762446700441936094341844739246701491961360505234382152206390825577848765149813266698206071501, 109464431735294159974378294274525657206318819420403729824301732669900016145525461205808884543364511930776254665221392803793527815537572044604556948966542735803855527632204173348346705081859987947224360357277306159043226383759325438531440117941819331729497009514666791332843453086877496404634366210511817347747, 167226938706190836553643155843119208197862054995067415100217554047405801743210621031327509390733457696587026428066379600702360290229750029561230959243313199878214139188992093824931076608557202587723520663649688259097308452343733278018286029658158983822856538007385804379149382950315116619472385431466470845443, 16595145287

### 3.3. Solving $j$ for $r$
We may also use the use the [<code>solve_j_for_r_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_r_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $j$ for $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791).

In [60]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r_mod_N;

result = solve_j_for_r_mod_N(j, m, l, g, N);

print("Solving j for r via [E22b] yielded:", result);

if result == r:
  print("[ OK ] The order r was successfully recovered.");
else:
  print("[FAIL] Failed to recover the order r.");

Solving j for r via [E22b] yielded: 1603061743479810892836166651413705796688192069345746840107261905949505124448112054940972632628687380029537398244200921428562042130647447733441594525359974923470200861221883375130437209870515949992543619137138803765362570342593639435711138895569739055732475508187915515328518477850533903258112597752074753260248643746280417896588905065644786553067840901362514139946208195069865545493598661474300720568895753261025341154269738000958015951286878150419134053850944713692350061673407550396714871175851895335102048925546145665285395635423787204409861172407292123681534688050931314883597816893850637542771602508826159874527944650625682520877208476000116298576306132191401631044747102548233215185844609506382556827435878870475175462110927104270659563554740193515092369051847340336131095940673098900240239764218425485058281656080856762463228478349196620949985472809807386915090980760170726442914967831025408986629291561744392144336334979976274950353334139628835531997787068

### 3.4. Splitting $N$ given $g$ and $r$ using Shor's original algorithm

We may then split $N$ given $g$ and $r$ using Shor's original algorithm in [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700).

To this end, we use the [<code>split_N_given_g_r(g, r, N)</code>](../docs/factoring/general/postprocessing/shor/split_N_given_g_r.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [61]:
from quaspy.factoring.general.postprocessing.shor import split_N_given_g_r;

result = split_N_given_g_r(g, r, N);

print("Attempting to split N using Shor's algorithm yielded:", result);

Attempting to split N using Shor's algorithm yielded: {580628326338291278781258643370619998195909114404305512646978285500641386862663977693393398141910619526230472922648457851256025355661938446783514998549851399547333689461651658813904541331033549685602400658292376079232182166936906569604941361869029256781382009277273599503640255354323463979674298354885041070013525125126026380566486431203747588544615984180703843816463733041185448326973832440499709319926434651264994204199413640492275039754906949711326207398268334480875851586522566265315136840996531863943155393684924248333321304744946245818736960367604676792741832113269523690401886076402011991093146137883762119610661436972990606557864332223712540750935952927802837624836892463237107042692304781511306144246930101336752906444042366029401094793642896969725134765853530963241193166170724825084299139466153114651479382034702550831929459648885581273380127868195312197432067636424704395575615075106992854509812025827939355121976386198091142715136858